In [10]:
import numpy as np
import dlib 
import cv2
import matplotlib.pyplot as plt
import time
import os
from collections import OrderedDict

import torch 
from torch import nn
from torch.utils.checkpoint import checkpoint
from torchsummary import summary
import sys
sys.path.append("./insightface/recognition/arcface_torch")

from backbones.iresnet import iresnet50
import mysql.connector

In [11]:
remote = mysql.connector.connect(
    host = "database-1.c7iiuw4kenou.ap-northeast-2.rds.amazonaws.com",
    user = "chillHome",
    password = "addinedu1!",
    database = "chillHome"
)

In [ ]:
cursor = remote.cursor()

: 

In [2]:
FACIAL_LANDMARKS_68_IDXS = OrderedDict([
	("mouth", (48, 68)),
	("inner_mouth", (60, 68)),
	("right_eyebrow", (17, 22)),
	("left_eyebrow", (22, 27)),
	("right_eye", (36, 42)),
	("left_eye", (42, 48)),
	("nose", (27, 36)),
	("jaw", (0, 17))
])

FACIAL_LANDMARKS_5_IDXS = OrderedDict([
	("right_eye", (2, 3)),
	("left_eye", (0, 1)),
	("nose", (4))
])

FACIAL_LANDMARKS_IDXS = FACIAL_LANDMARKS_68_IDXS

def rect_to_bb(rect):
	x = rect.left()
	y = rect.top()
	w = rect.right() - x
	h = rect.bottom() - y
	return (x, y, w, h)

def shape_to_np(shape, dtype="int"):
	coords = np.zeros((shape.num_parts, 2), dtype=dtype)

	for i in range(0, shape.num_parts):
		coords[i] = (shape.part(i).x, shape.part(i).y)

	return coords

In [ ]:
class FaceAligner:
    def __init__(self, predictor, desiredLeftEye=(0.35, 0.35),
        desiredFaceWidth=256, desiredFaceHeight=None):

        self.predictor = predictor
        self.desiredLeftEye = desiredLeftEye
        self.desiredFaceWidth = desiredFaceWidth
        self.desiredFaceHeight = desiredFaceHeight


        if self.desiredFaceHeight is None:
            self.desiredFaceHeight = self.desiredFaceWidth

    def align(self, image, gray, rect):
        shape = self.predictor(gray, rect)
        shape = shape_to_np(shape)

        if (len(shape)==68):
            (lStart, lEnd) = FACIAL_LANDMARKS_68_IDXS["left_eye"]
            (rStart, rEnd) = FACIAL_LANDMARKS_68_IDXS["right_eye"]
        else:
            (lStart, lEnd) = FACIAL_LANDMARKS_5_IDXS["left_eye"]
            (rStart, rEnd) = FACIAL_LANDMARKS_5_IDXS["right_eye"]
            
        leftEyePts = shape[lStart:lEnd]
        rightEyePts = shape[rStart:rEnd]

        leftEyeCenter = leftEyePts.mean(axis=0).astype("int")
        rightEyeCenter = rightEyePts.mean(axis=0).astype("int")

        dY = rightEyeCenter[1] - leftEyeCenter[1]
        dX = rightEyeCenter[0] - leftEyeCenter[0]
        angle = np.degrees(np.arctan2(dY, dX)) - 180

        desiredRightEyeX = 1.0 - self.desiredLeftEye[0]

        dist = np.sqrt((dX ** 2) + (dY ** 2))
        desiredDist = (desiredRightEyeX - self.desiredLeftEye[0])
        desiredDist *= self.desiredFaceWidth
        scale = desiredDist / dist

        eyesCenter = (int((leftEyeCenter[0] + rightEyeCenter[0]) // 2),
            int((leftEyeCenter[1] + rightEyeCenter[1]) // 2))

        M = cv2.getRotationMatrix2D(eyesCenter, angle, scale)

        tX = self.desiredFaceWidth * 0.5
        tY = self.desiredFaceHeight * self.desiredLeftEye[1]
        M[0, 2] += (tX - eyesCenter[0])
        M[1, 2] += (tY - eyesCenter[1])

        (w, h) = (self.desiredFaceWidth, self.desiredFaceHeight)
        output = cv2.warpAffine(image, M, (w, h),
            flags=cv2.INTER_CUBIC)

        return output

In [5]:
predictor_file = "./model/shape_predictor_68_face_landmarks.dat"
face_detector = dlib.get_frontal_face_detector()
shape_predictor = dlib.shape_predictor(predictor_file)
fa = FaceAligner(shape_predictor, desiredFaceWidth=112)

In [6]:
def l2_norm(x, axis=1):
    norm = np.linalg.norm(x, axis=axis, keepdims=True)
    return x / norm

In [7]:
def face_embedding(model, img, dsize=112, device='cuda'):
    img = cv2.resize(img, (dsize,dsize))
    img = np.transpose(img, (2, 0, 1))
    img = torch.from_numpy(img).unsqueeze(0).float()
    img.div_(255).sub_(0.5).div_(0.5)
    img = img.to(device)
    embed = model(img).detach().cpu().numpy()
    return l2_norm(embed)

In [8]:
weight_path = "./model/face_recognition.pt"
device = torch.device("mps")
model = iresnet50().to(device)
model.load_state_dict(torch.load(weight_path, map_location = device, weights_only=True))
model.eval()
print("Model Loaded!")

Model Loaded!


In [ ]:
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    raise RuntimeError("카메라가 열리지 않습니다.")

try:
    while True:
        ret, frame = cap.read()

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        face_detection = face_detector(frame, 2)
        
        if len(face_detection) > 0:
            f = face_detection[0]
            faceAligned = fa.align(frame, gray, f)
            known_embed = face_embedding(model, faceAligned, 112)
            cursor.execute(f"INSERT INTO faceEmbeddings (userId, embedding) VALUES(1, {known_embed})")
            break

        cv2.imshow('frame', gray)

        if cv2.waitKey(1) == 27:
            break

finally:
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    raise RuntimeError("카메라가 열리지 않습니다.")

try:
    while True:
        ret, frame = cap.read()

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        face_detection = face_detector(frame, 2)
        
        if len(face_detection) > 0:
            f = face_detection[0]
            faceAligned = fa.align(frame, gray, f)
            known_embed = face_embedding(model, faceAligned, 112)
            cursor.execute(f"SELECT count(*) as cnt FROM faceEmbeddings WHERE embedding = {known_embed}")
            
            result = cursor.fetchone()

            if result[0] > 0:
                print("인증 성공")
            else:
                print("인증 실패")

        cv2.imshow('frame', gray)

        if cv2.waitKey(1) == 27:
            break

finally:
    cap.release()
    cv2.destroyAllWindows()